In [1]:
import pandas as pd
import numpy as np

df_jan = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_feb = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [2]:
df_jan.shape
df_feb.shape

(2913955, 19)

In [3]:
df_jan.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [4]:
len(df_jan.columns)

19

In [5]:
df_jan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [6]:
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [7]:
# calculate the duration from df_jan
df_jan['duration'] = (df_jan['tpep_dropoff_datetime'] - df_jan['tpep_pickup_datetime']).dt.total_seconds() / 60

# calculate the duration from df_feb
df_feb['duration'] = (df_feb['tpep_dropoff_datetime'] - df_feb['tpep_pickup_datetime']).dt.total_seconds() / 60

In [8]:
df_jan['duration'].std()

42.594351241920904

In [9]:
len(df_jan['duration'][(df_jan['duration'] >= 1) & (df_jan['duration'] <=60)]) / len(df_jan['duration'])

0.9812202822125979

In [10]:
filtered_df_jan = df_jan[(df_jan['duration'] >= 1) & (df_jan['duration'] <= 60)].copy()

In [11]:
filtered_df_feb = df_feb[(df_feb['duration'] >= 1) & (df_feb['duration'] <= 60)].copy()

In [12]:
filtered_df_jan['PULocationID'] = filtered_df_jan['PULocationID'].astype(str)
filtered_df_jan['DOLocationID'] = filtered_df_jan['DOLocationID'].astype(str)

In [13]:
pickup_dropoff_dicts = filtered_df_jan[['PULocationID', 'DOLocationID']].to_dict(orient='records')

In [14]:
from sklearn.feature_extraction import DictVectorizer

vectorizer = DictVectorizer() 
feature_matrix = vectorizer.fit_transform(pickup_dropoff_dicts)
print(type(feature_matrix))

<class 'scipy.sparse.csr.csr_matrix'>


In [15]:
feature_matrix.shape

(3009173, 515)

In [16]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [17]:
X = feature_matrix  
y = filtered_df_jan['duration']  

In [18]:
model = LinearRegression()  
model.fit(X, y)  

LinearRegression()

In [19]:
y_pred = model.predict(X)  
rmse = np.sqrt(mean_squared_error(y, y_pred))  
print("RMSE on training data:", rmse)

RMSE on training data: 7.649261027919939


In [20]:
filtered_df_feb['PULocationID'] = filtered_df_feb['PULocationID'].astype(str)
filtered_df_feb['DOLocationID'] = filtered_df_feb['DOLocationID'].astype(str)
validation_dicts = filtered_df_feb[['PULocationID', 'DOLocationID']].to_dict(orient='records')

X_validation = vectorizer.transform(validation_dicts)
X_validation

<2855951x515 sparse matrix of type '<class 'numpy.float64'>'
	with 5711894 stored elements in Compressed Sparse Row format>

In [21]:

y_validation = filtered_df_feb['duration']
len(y_validation)

2855951

In [22]:
y_pred_validation = model.predict(X_validation)

In [23]:
rmse_validation = np.sqrt(mean_squared_error(y_validation, y_pred_validation))
print("RMSE on validation data:", rmse_validation)

RMSE on validation data: 7.811832638273232
